In [1]:
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
plt.style.use ('ggplot')
import seaborn as sns
%matplotlib inline
sns.set_style(style="whitegrid")

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import NMF
from sklearn.feature_extraction import text

from bs4 import BeautifulSoup

import nltk, string

In [2]:
df = pd.read_csv('data/modeling_ready_microwave1')

In [4]:
df.head()

,customer_id,review_id,product_id,star_rating,helpful_votes,total_votes,verified_purchase,review_headline,review_body,review_date,review_wordcount,clean_review
0,44300577,R2ZU11YALTJNZX,B0009KMYHI,1.0,0.0,0.0,Y,I have had this microwave for just over 3 year...,I have had this microwave for just over 3 year...,2015-08-31,91,I have had this microwave for just over 3 year...
1,50952586,RVBITZNBVJ8AI,B0009KMYHI,5.0,0.0,0.0,Y,It takes a good bit of room on the counter,Man this thing cooks evenly and fast! It takes...,2015-08-30,23,Man this thing cooks evenly and fast! It takes...
2,31144708,R44LZL0OR5EWP,B0009KMYDM,5.0,0.0,0.0,Y,Five Stars,Works great! Have had it over a year and no su...,2015-08-30,17,Works great! Have had it over a year and no su...
3,14667788,RPWWYNUD9X64U,B0009KMYHI,5.0,0.0,0.0,Y,... received this several years ago and it sti...,I ordered and received this several years ago ...,2015-08-29,13,I ordered and received this several years ago ...
4,25468208,RS76DCCGM685B,B0009KMYGY,1.0,3.0,3.0,Y,Avoid Panasonic - Doors Don't Close,I have had this microwave for just over 2 year...,2015-08-28,131,I have had this microwave for just over 2 year...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1875 entries, 0 to 1874
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   customer_id        1875 non-null   int64  
 1   review_id          1875 non-null   object 
 2   product_id         1875 non-null   object 
 3   star_rating        1875 non-null   float64
 4   helpful_votes      1875 non-null   float64
 5   total_votes        1875 non-null   float64
 6   verified_purchase  1875 non-null   object 
 7   review_headline    1875 non-null   object 
 8   review_body        1875 non-null   object 
 9   review_date        1875 non-null   object 
 10  review_wordcount   1875 non-null   int64  
 11  clean_review       1875 non-null   object 
dtypes: float64(3), int64(2), object(7)
memory usage: 175.9+ KB


In [3]:
df.review_date = pd.to_datetime(df.review_date)

In [7]:
df.describe()

,customer_id,star_rating,helpful_votes,total_votes,review_wordcount
count,1.875000e+03,1875.000000,1875.000000,1875.000000,1875.000000
mean,3.209401e+07,3.850133,3.107733,3.628267,80.606933
std,1.476223e+07,1.496151,20.062047,20.926332,97.269705
min,8.722000e+04,1.000000,0.000000,0.000000,1.000000
25%,1.748780e+07,3.000000,0.000000,0.000000,28.000000
50%,3.303278e+07,5.000000,1.000000,1.000000,51.000000
75%,4.624525e+07,5.000000,2.000000,2.000000,94.000000
max,5.309426e+07,5.000000,589.000000,599.000000,1152.000000


In [4]:
punct =[]
punct += list(string.punctuation)
punct += '’'
punct.remove("'")
def remove_punctuations(text):
    for punctuation in punct:
        text = text.replace(punctuation, ' ')
    return text

In [5]:
def nlp(df):
    # lowercase everything
    # get rid of '\n' from whitespace
    # regex remove hyperlinks
    # remove punctuation
    # remove ' s ' from removing punctuation
    
    # lowercase everything
    df['token'] = df['clean_review'].apply(lambda x: x.lower())
    # get rid of '\n' from whitespace 
    df['token'] = df['token'].apply(lambda x: x.replace('\n', ' '))
    # regex remove hyperlinks
    df['token'] = df['token'].str.replace('http\S+|www.\S+', '', case=False)
    # remove punctuations
    df['token'] = df['token'].apply(remove_punctuations)
    # remove ' s ' that was created after removing punctuations
    df['token'] = df['token'].apply(lambda x: str(x).replace(" s ", " "))
    return df

In [11]:
nlp(df)

,customer_id,review_id,product_id,star_rating,helpful_votes,total_votes,verified_purchase,review_headline,review_body,review_date,review_wordcount,clean_review,token
0,44300577,R2ZU11YALTJNZX,B0009KMYHI,1.0,0.0,0.0,Y,I have had this microwave for just over 3 year...,I have had this microwave for just over 3 year...,2015-08-31,91,I have had this microwave for just over 3 year...,i have had this microwave for just over 3 year...
1,50952586,RVBITZNBVJ8AI,B0009KMYHI,5.0,0.0,0.0,Y,It takes a good bit of room on the counter,Man this thing cooks evenly and fast! It takes...,2015-08-30,23,Man this thing cooks evenly and fast! It takes...,man this thing cooks evenly and fast it takes...
2,31144708,R44LZL0OR5EWP,B0009KMYDM,5.0,0.0,0.0,Y,Five Stars,Works great! Have had it over a year and no su...,2015-08-30,17,Works great! Have had it over a year and no su...,works great have had it over a year and no su...
3,14667788,RPWWYNUD9X64U,B0009KMYHI,5.0,0.0,0.0,Y,... received this several years ago and it sti...,I ordered and received this several years ago ...,2015-08-29,13,I ordered and received this several years ago ...,i ordered and received this several years ago ...
4,25468208,RS76DCCGM685B,B0009KMYGY,1.0,3.0,3.0,Y,Avoid Panasonic - Doors Don't Close,I have had this microwave for just over 2 year...,2015-08-28,131,I have had this microwave for just over 2 year...,i have had this microwave for just over 2 year...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1870,33705501,R34GY91QSWWUE4,B0009KMYGY,4.0,60.0,64.0,N,Just right for me...,The Panasonic NN-H765WF was exactly what I was...,2006-03-23,74,The Panasonic NN-H765WF was exactly what I was...,the panasonic nn h765wf was exactly what i was...
1871,39268146,R7N1LRCV8GU6L,B0009KMYHI,5.0,5.0,10.0,Y,good looking-good cooking,this microwave cooks food very evenly-no need ...,2006-03-19,21,this microwave cooks food very evenly-no need ...,this microwave cooks food very evenly no need ...
1872,19314267,R15KWS9QSI7URC,B0009KMYDM,5.0,12.0,13.0,Y,Panasonic NN-H965WF Luxury full-size 2.2 cu.ft...,"its a great microwave , i use it every day , i...",2006-03-16,67,"its a great microwave , i use it every day , i...",its a great microwave i use it every day i...
1873,11400155,RK5HSUM1GNRH8,B0009KMYDM,5.0,4.0,9.0,N,Panasonic Microwave Oven NN-965WF,Have had it for only a short time but it has p...,2006-02-24,20,Have had it for only a short time but it has p...,have had it for only a short time but it has p...


In [6]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

In [7]:
stop_words = text.ENGLISH_STOP_WORDS.union(['panasonic', 'really', 'husband', 
                                            'thanks', 'thank', 'ha', 'just', 'thing', 
                                            'did', 'nn', 'wa', 'yr', 'u', 'say', 'doe'])

In [8]:
vectorizer = TfidfVectorizer(lowercase=True,
                             tokenizer = lemmatize_text,
                            stop_words=stop_words,
                            max_features = 1000,
                            min_df = 3,
                            max_df = .85)

In [12]:
X = df['token']

In [13]:
X = vectorizer.fit_transform(X)

/home/eric/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [14]:
X.shape

(1875, 1000)

In [16]:
df.head()

,customer_id,review_id,product_id,star_rating,helpful_votes,total_votes,verified_purchase,review_headline,review_body,review_date,review_wordcount,clean_review,token
0,44300577,R2ZU11YALTJNZX,B0009KMYHI,1.0,0.0,0.0,Y,I have had this microwave for just over 3 year...,I have had this microwave for just over 3 year...,2015-08-31,91,I have had this microwave for just over 3 year...,i have had this microwave for just over 3 year...
1,50952586,RVBITZNBVJ8AI,B0009KMYHI,5.0,0.0,0.0,Y,It takes a good bit of room on the counter,Man this thing cooks evenly and fast! It takes...,2015-08-30,23,Man this thing cooks evenly and fast! It takes...,man this thing cooks evenly and fast it takes...
2,31144708,R44LZL0OR5EWP,B0009KMYDM,5.0,0.0,0.0,Y,Five Stars,Works great! Have had it over a year and no su...,2015-08-30,17,Works great! Have had it over a year and no su...,works great have had it over a year and no su...
3,14667788,RPWWYNUD9X64U,B0009KMYHI,5.0,0.0,0.0,Y,... received this several years ago and it sti...,I ordered and received this several years ago ...,2015-08-29,13,I ordered and received this several years ago ...,i ordered and received this several years ago ...
4,25468208,RS76DCCGM685B,B0009KMYGY,1.0,3.0,3.0,Y,Avoid Panasonic - Doors Don't Close,I have had this microwave for just over 2 year...,2015-08-28,131,I have had this microwave for just over 2 year...,i have had this microwave for just over 2 year...


In [15]:
nmf = NMF(
    n_components=30,
    init='nndsvd'
).fit(X)

nmf_output = nmf.fit_transform(X)

In [16]:
def show_topics(vectorizer=vectorizer, lda_model=nmf, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

In [17]:
topic_keywords = show_topics(vectorizer=vectorizer, lda_model=nmf, n_words=8)        

In [18]:
topic_keywords

[array(['microwave', 'best', 'looking', 'used', 'owned', 'problem',
        'noise', 'need'], dtype='<U13'),
 array(['amazon', 'service', 'day', 'repair', 'customer', 'shipping',
        'warranty', 'center'], dtype='<U13'),
 array(['great', 'price', 'item', 'feature', 'lot', 'size', 'amazon',
        'product'], dtype='<U13'),
 array(['door', 'open', 'latch', 'close', 'push', 'hard', 'shut', 'slam'],
       dtype='<U13'),
 array(['use', 'easy', 'clean', 'operate', 'pleased', 'room', 'heat',
        'understand'], dtype='<U13'),
 array(['oven', 'watt', 'cooking', 'display', 'inverter', 'owned',
        'problem', '1250'], dtype='<U13'),
 array(['love', 'big', 'guess', 'gift', 'christmas', 'meat', 'setting',
        'mom'], dtype='<U13'),
 array(['power', 'level', 'inverter', 'setting', 'watt', 'lot', 'lower',
        'set'], dtype='<U13'),
 array(['old', 'new', 'replaced', 'faster', 'year', 'sharp', 'replace',
        'heat'], dtype='<U13'),
 array(['good', 'price', 'got', 'far', 'item

In [19]:
# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]

Topics_theme = range(30)
df_topic_keywords['topic_theme'] = Topics_theme
df_topic_keywords.set_index('topic_theme', inplace=True)
df_topic_keywords.T

topic_theme,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
Word 0,microwave,amazon,great,door,use,oven,love,power,old,good,...,nice,time,large,happy,powerful,popcorn,working,recommend,month,bought
Word 1,best,service,price,open,easy,watt,big,level,new,price,...,big,cooking,fit,purchase,big,button,quit,highly,buy,second
Word 2,looking,day,item,latch,clean,cooking,guess,inverter,replaced,got,...,expected,long,space,far,large,setting,stopped,larger,worked,gift
Word 3,used,repair,feature,close,operate,display,gift,setting,faster,far,...,look,used,size,i'm,sure,bag,completely,looking,warranty,replace
Word 4,owned,customer,lot,push,pleased,inverter,christmas,watt,year,item,...,feature,item,kitchen,quite,loud,push,week,pleased,6,ago
Word 5,problem,shipping,size,hard,room,owned,meat,lot,sharp,value,...,job,start,counter,heat,spacious,pop,got,amazing,i've,2013
Word 6,noise,warranty,amazon,shut,heat,problem,setting,lower,replace,look,...,size,second,needed,dish,tray,don't,far,size,money,review
Word 7,need,center,product,slam,understand,1250,mom,set,heat,quality,...,complaint,minute,need,inside,food,minute,stop,definitely,broke,office


In [20]:
stop_words = ['panasonic', 'really', 'husband', 'thanks', 'thank', 'ha', 
              'just', 'thing', 'did', 'nn', 'wa', 'yr', 'u', 'say', 'doe',
             'mom', 'christmas', 'gift', 'got', 'way', 'le']

In [22]:
topic_keywords

[array(['microwave', 'best', 'looking', 'used', 'owned', 'problem',
        'noise', 'need'], dtype='<U13'),
 array(['amazon', 'service', 'day', 'repair', 'customer', 'shipping',
        'warranty', 'center'], dtype='<U13'),
 array(['great', 'price', 'item', 'feature', 'lot', 'size', 'amazon',
        'product'], dtype='<U13'),
 array(['door', 'open', 'latch', 'close', 'push', 'hard', 'shut', 'slam'],
       dtype='<U13'),
 array(['use', 'easy', 'clean', 'operate', 'pleased', 'room', 'heat',
        'understand'], dtype='<U13'),
 array(['oven', 'watt', 'cooking', 'display', 'inverter', 'owned',
        'problem', '1250'], dtype='<U13'),
 array(['love', 'big', 'guess', 'gift', 'christmas', 'meat', 'setting',
        'mom'], dtype='<U13'),
 array(['power', 'level', 'inverter', 'setting', 'watt', 'lot', 'lower',
        'set'], dtype='<U13'),
 array(['old', 'new', 'replaced', 'faster', 'year', 'sharp', 'replace',
        'heat'], dtype='<U13'),
 array(['good', 'price', 'got', 'far', 'item